In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from dotenv import load_dotenv
from typing import TypedDict
import os

In [ ]:
load_dotenv()

In [ ]:
blog_model = ChatOpenAI(
    base_url= "https://api.ai.it.ufl.edu",
    model = "llama-3.1-70b-instruct",
    temperature=0.1,
    api_key=os.getenv("NAVIGATOR_TOOLKIT_API_KEY")
)

def initialize_summary_model(max_tokens = 500):
    summary_model = ChatOpenAI(
        base_url= "https://api.ai.it.ufl.edu",
        model = "llama-3.1-8b-instruct",
        temperature=0.1,
        api_key=os.getenv("NAVIGATOR_TOOLKIT_API_KEY"),
        max_tokens=max_tokens
    )
    return summary_model

In [ ]:
class SummarizerLLM_QA_State(TypedDict):
    question : str
    outline : str
    topic : str
    blog : str
    summary_max_tokens : int
    answer : str
    summary : str

In [ ]:
def generate_outline(state: SummarizerLLM_QA_State) -> SummarizerLLM_QA_State:
    topic = state["topic"]
    question = state["question"]
    messages = [
        SystemMessage(content="You are a helpful assistant that generate organized outline for a blog article based on a topic and a question."),
        HumanMessage(content=f"Topic: {topic}\nQuestion: {question}")
    ]
    response = blog_model.invoke(messages)
    state["outline"] = response.content
    return state


def generate_blog(state: SummarizerLLM_QA_State) -> SummarizerLLM_QA_State:
    topic = state["topic"]
    question = state["question"]
    outline = state["outline"]
    messages = [
        SystemMessage(content="You are a helpful assistant that generate long blogs articles based on a topic and a question. Your response should be organized based on the outline provided."),
        HumanMessage(content=f"Topic: {topic}\nQuestion: {question}, \nOutline: {outline}")
    ]
    response = blog_model.invoke(messages)
    state["blog"] = response.content
    return state

def generate_summary(state: SummarizerLLM_QA_State) -> SummarizerLLM_QA_State:
    blog = state["blog"]
    outline = state["outline"]
    summary_max_tokens = state["summary_max_tokens"]

    summary_model = initialize_summary_model(summary_max_tokens)
    print(f"Summary model initialized with max tokens: {summary_max_tokens}")

    messages = [
        SystemMessage(content="You are a helpful assistant that summarize a blog article staying context aware about the outline."),
        HumanMessage(content=f"Blog: {blog}, \nOutline: {outline}")
    ]
    response = summary_model.invoke(messages)
    state["summary"] = response.content

    return state

In [ ]:
# Define the graph

# START ---> Generate outline ---> Generate blog ---> Generate summary ---> END
graph = StateGraph(SummarizerLLM_QA_State)

# Add nodes to graph
graph.add_node("generate_outline", generate_outline)
graph.add_node("generate_blog", generate_blog)
graph.add_node("generate_summary", generate_summary)

# Add edges to graph
graph.add_edge(START, "generate_outline")
graph.add_edge("generate_outline", "generate_blog")
graph.add_edge("generate_blog", "generate_summary")
graph.add_edge("generate_summary", END)

# Compile the graph
workflow = graph.compile()
workflow


In [ ]:
initial_state = {
    "question": "How to pick your first DSLR camera on a tight budget.",
    "topic": "Camera",
    "summary_max_tokens": 1000
}

final_state = workflow.invoke(initial_state, verbose=True)

In [ ]:
print("========================")
print("Outline")
print(final_state["outline"])

print("Blog:")
print(final_state["blog"])

print("Summary:")
print(final_state["summary"])